In [1]:
from pathlib import Path; import os
cwd_path = Path.cwd(); set_path = str(cwd_path.parent); os.chdir(set_path)

In [2]:
#from tensorflow_addons.layers import WeightNormalization

In [3]:
import numpy as np

In [4]:
from tensorflow_probability import layers as tfp_layers
WeightNormalization = tfp_layers.weight_norm.WeightNorm
import tensorflow as tf

In [5]:
from tensorflow.keras.layers import Dense, Flatten, Layer, Reshape, Concatenate

In [6]:
from Inverse_Autoregressive_Flow.AutoregressiveNN.Input_layer import Autoregressive_input_layer

In [7]:
input_layer = Autoregressive_input_layer(3, 4, units=7)
latent_z = np.array([1, 1000, 1005], dtype="float32")[np.newaxis, :]
h = np.ones((4,), dtype="float32")[np.newaxis, :]
print(input_layer([latent_z, h]))

tf.Tensor(
[[ 5.0343215e-03 -1.3214946e-02  5.3657074e-02 -7.5748801e-02
  -1.6861320e-02 -1.0000000e+00  9.1054688e+01]], shape=(1, 7), dtype=float32)


In [8]:
# Gives an error
"""
input_layer = WeightNormalization(Autoregressive_input_layer(3, 4, units=7))
latent_z = np.array([1, 1000, 1005], dtype="float32")[np.newaxis, :]
h = np.ones((4,), dtype="float32")[np.newaxis, :]
print(input_layer([latent_z, h]))
"""
None

In [9]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.biases = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel) + self.biases

In [37]:
Linear()(latent_z)

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ -82.502045 ,    2.1578898,   -4.050124 ,  -26.298466 ,
          29.399246 ,  170.78648  ,  -92.37528  ,  -38.358284 ,
           8.093197 ,  -82.31457  ,   48.380844 ,  -29.157846 ,
          29.094372 ,  -22.753048 , -118.95863  ,   62.945435 ,
          -7.698193 ,  -36.90656  ,  -16.66807  ,  -71.744    ,
          15.480982 ,  -31.885298 ,   -5.15213  ,   82.992584 ,
          -5.1770186, -174.69043  ,   95.22082  ,  105.72761  ,
          12.130077 ,   -5.6468024,  -36.42094  ,   82.22339  ]],
      dtype=float32)>

In [39]:
lay = WeightNormalization(Linear())
lay(latent_z)

TypeError: 'NoneType' object is not subscriptable

In [25]:
class Autoregressive_input_layer(Layer):
    """
    Autoregressive inputs refer to inputs were we need to maintain autoregressive structure as in MADE paper
    Specifically here the dimension of the autoregressive_input_dim is the latent representation dimension
    non_autoregressive inputs are fully connected (this represents h in IAF paper)

    """
    def __init__(self, autoregressive_input_dim, non_autoregressive_input_dim, units=64):
        super(Autoregressive_input_layer, self).__init__()
        assert units >= autoregressive_input_dim
        self.autoregressive_input_dim = autoregressive_input_dim
        self.non_autoregressive_input_dim = non_autoregressive_input_dim
        self.units = units
    
    def build(self, input_shape):
        autoregressive_input_dim = self.autoregressive_input_dim #input_shape[0][-1]
        non_autoregressive_input_dim = self.non_autoregressive_input_dim #input_shape[1][-1]
        self.kernel = self.add_weight(
        shape=(autoregressive_input_dim, self.units),
        initializer="random_normal",
        trainable=True,
        )
        # masking
        self.autoregressive_weights_mask = np.zeros((autoregressive_input_dim, self.units))
        nodes_per_latent_representation_dim = self.units / autoregressive_input_dim
        for i in range(autoregressive_input_dim):
            for j in range(self.units):
                units_corresponding_max_autoregressive_input_index = j//nodes_per_latent_representation_dim
                if units_corresponding_max_autoregressive_input_index > i:
                    self.autoregressive_weights_mask[i, j] = 1
        self.autoregressive_weights_mask = tf.convert_to_tensor(self.autoregressive_weights_mask, dtype="float32")
        
        self.non_autoregressive_weights = self.add_weight(shape=(non_autoregressive_input_dim, self.units),
                                                                 initializer="random_normal",
                                                                 trainable=True, dtype="float32")
        
        # biases
        self.biases = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True)


    def call(self, inputs):
        h_non_autoregressive = inputs[:, self.autoregressive_input_dim:]
        z_autoregressive = inputs[:, :self.autoregressive_input_dim]
        x = tf.matmul(z_autoregressive, self.autoregressive_weights*self.autoregressive_weights_mask) + \
            tf.matmul(h_non_autoregressive, self.non_autoregressive_weights) \
            + self.biases
        #return tf.nn.relu(x)
        return tf.nn.elu(x)

In [26]:
input_layer = Autoregressive_input_layer(3, 4, units=7)
latent_z = np.array([1, 1000, 1005], dtype="float32")[np.newaxis, :]
h = np.ones((4,), dtype="float32")[np.newaxis, :]
concat = tf.keras.layers.Concatenate()([latent_z, h])
print(input_layer(concat))

tf.Tensor(
[[ 9.8826647e-02  5.2792039e-03  2.7916100e-02 -6.9247961e-02
  -1.2243986e-01 -1.0000000e+00  5.8842025e+00]], shape=(1, 7), dtype=float32)


In [29]:
input_layer = WeightNormalization(Autoregressive_input_layer(3, 4, units=7))
latent_z = np.array([1, 1000, 1005], dtype="float32")[np.newaxis, :]
h = np.ones((4,), dtype="float32")[np.newaxis, :]
concat = tf.keras.layers.Concatenate()([latent_z, h])
print(input_layer(concat))

ValueError: `WeightNorm` must wrap a layer that contains a `kernel` for weights

# Dense

In [ ]:
x = WeightNormalization(tf.keras.layers.Dense(10, activation="elu", name="layer1"),
                                            data_init=True)

In [ ]:
x.trainable_variables

In [ ]:
test_inputs = np.array([[1.0,2.0]])

In [ ]:
x(test_inputs)

# Overwrite Dense?

In [ ]:
import tensorflow as tf

In [ ]:
Dense(1).get_config()

In [ ]:
class Dense(tf.keras.layers.Dense):
    def __init__(self, *args, **kwargs):
        super(Dense, self).__init__(*args, **kwargs)
        self.dummy = 1e6
    
    def call(self, inputs):
        return (tf.matmul(inputs, self.kernel) + self.bias) * self.dummy

In [ ]:
Dense(3).get_config()

In [ ]:
overwritten = WeightNormalization(Dense(3),
                                            data_init=True)

In [ ]:
overwritten(test_inputs)

# Custom

In [ ]:
weight_initialiser = tf.random_normal_initializer()
self.autoregressive_weights = \
    tf.Variable(initial_value=weight_initialiser(shape=(autoregressive_input_dim, units),
                dtype="float32"), trainable=True)
self.autoregressive_weights_mask = np.zeros((autoregressive_input_dim, units))
# TODO can do this without a lengthy loop

nodes_per_latent_representation_dim = units / autoregressive_input_dim
for i in range(autoregressive_input_dim):
    for j in range(units):
        units_corresponding_max_autoregressive_input_index = j//nodes_per_latent_representation_dim
        if units_corresponding_max_autoregressive_input_index > i:
            self.autoregressive_weights_mask[i, j] = 1
self.autoregressive_weights_mask = tf.convert_to_tensor(self.autoregressive_weights_mask, dtype="float32")
#self.L_mask = np.zeros((autoregressive_input_dim, units)).astype("float32")
#self.L_mask = np.tril(self.L_mask, k=-1) # we can do this if units=autoregressive_input_dim

self.non_autoregressive_weights = \
    tf.Variable(initial_value=weight_initialiser(shape=(non_autoregressive_input_dim, units),
                                                 dtype="float32"), trainable=True)
self.biases = tf.Variable(initial_value=tf.zeros_initializer()(shape=(units, ), dtype="float32"),
                          trainable=True)

In [ ]:
class Autoregressive_input_layer(tf.keras.layers.Dense):
    """
    Autoregressive inputs refer to inputs were we need to maintain autoregressive structure as in MADE paper
    Specifically here the dimension of the autoregressive_input_dim is the latent representation dimension
    non_autoregressive inputs are fully connected (this represents h in IAF paper)

    """
    def __init__(self, autoregressive_input_dim, non_autoregressive_input_dim, units=64):
        super(Autoregressive_input_layer, self).__init__()
        assert units >= autoregressive_input_dim
        self.autoregressive_input_dim = autoregressive_input_dim
        self.non_autoregressive_input_dim = non_autoregressive_input_dim
        self.units = units
    
    def build(input_shape):
                self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.bias = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )


    def call(self, inputs):
        z_autoregressive, h_non_autoregressive = inputs
        x = tf.matmul(z_autoregressive, self.autoregressive_weights*self.autoregressive_weights_mask) + \
            tf.matmul(h_non_autoregressive, self.non_autoregressive_weights) \
            + self.biases
        #return tf.nn.relu(x)
        return tf.nn.elu(x)

In [ ]:
class Dense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Dense, self).__init__(**kwargs)
        self.units = units
        self.dogfish  = 2

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.bias = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.bias

    def get_config(self):
        config = super(Dense, self).get_config()
        config.update({"units": self.units})
        return config

In [ ]:
x = WeightNormalization(Dense())

In [ ]:
x(test_inputs)

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Layer, Reshape, Concatenate
import tensorflow as tf
import numpy as np

class Autoregressive_input_layer(Layer):
    """
    Autoregressive inputs refer to inputs were we need to maintain autoregressive structure as in MADE paper
    Specifically here the dimension of the autoregressive_input_dim is the latent representation dimension
    non_autoregressive inputs are fully connected (this represents h in IAF paper)

    """
    def __init__(self, autoregressive_input_dim, non_autoregressive_input_dim, units=64):
        super(Autoregressive_input_layer, self).__init__()
        assert units >= autoregressive_input_dim
        weight_initialiser = tf.random_normal_initializer()
        self.autoregressive_weights = \
            tf.Variable(initial_value=weight_initialiser(shape=(autoregressive_input_dim, units),
                        dtype="float32"), trainable=True)
        self.autoregressive_weights_mask = np.zeros((autoregressive_input_dim, units))
        nodes_per_latent_representation_dim = units / autoregressive_input_dim
        for i in range(autoregressive_input_dim):
            for j in range(units):
                units_corresponding_max_autoregressive_input_index = j//nodes_per_latent_representation_dim
                if units_corresponding_max_autoregressive_input_index > i:
                    self.autoregressive_weights_mask[i, j] = 1
        self.autoregressive_weights_mask = tf.convert_to_tensor(self.autoregressive_weights_mask, dtype="float32")


        self.non_autoregressive_weights = \
            tf.Variable(initial_value=weight_initialiser(shape=(non_autoregressive_input_dim, units),
                                                         dtype="float32"), trainable=True)
        self.biases = tf.Variable(initial_value=tf.zeros_initializer()(shape=(units, ), dtype="float32"),
                                  trainable=True)

    def call(self, inputs):
        z_autoregressive, h_non_autoregressive = inputs
        x = tf.matmul(z_autoregressive, self.autoregressive_weights*self.autoregressive_weights_mask) + \
            tf.matmul(h_non_autoregressive, self.non_autoregressive_weights) \
            + self.biases
        #return tf.nn.relu(x)
        return tf.nn.elu(x)

if __name__ == "__main__":
    # simple example with latent dim of 2
    input_layer = Autoregressive_input_layer(3, 4, units=7)
    print(input_layer.autoregressive_weights_mask)
    latent_z = np.array([1, 1000, 1005], dtype="float32")[np.newaxis, :]
    h = np.ones((4,), dtype="float32")[np.newaxis, :]
    print(input_layer([latent_z, h]))